<a href="https://www.kaggle.com/code/sergeypolivin/language-recognition-using-bert?scriptVersionId=218144923" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    logging, 
    pipeline,
)

DATA_DIR = "/kaggle/input/language-detection/"
MODEL_NAME = "bert-base-multilingual-uncased"
RANDOM_STATE = 12345
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 1
LEARNING_RATE = 5e-5

torch.backends.cudnn.deterministic = True
torch.manual_seed(RANDOM_STATE)
logging.set_verbosity_error()

In [ ]:
# Displaying the device used for fune-tuning BERT
print(f"Device used: {DEVICE}")

# Loading data

In [ ]:
# Loading the data
lang_data = pd.read_csv(DATA_DIR + "Language Detection.csv")

# Displaying random 30 rows
lang_data.sample(n=30, random_state=RANDOM_STATE)

In [ ]:
# Displaying a concise summary of the data
lang_data.info()

# Data exploration

In [ ]:
# Plotting the structure of the target variable
lang_counts = lang_data["Language"].value_counts(normalize=True)
lang_counts.plot(kind="bar")
plt.title("Shares of objects in each language class", fontsize=15)
plt.ylabel("Proportion of objects")
plt.tight_layout()
plt.show()

# Encoding labels

In [ ]:
# Retrieving the text data
texts_data = lang_data["Text"].values.astype("U")

# Retrieving the labels data
labels_data = lang_data["Language"].values

In [ ]:
# Instantiating the LabelEncoder object
label_encoder = LabelEncoder()

# Encoding the labels
labels_data_encoded = label_encoder.fit_transform(labels_data)
class_names = label_encoder.classes_

# Displaying the encoding results
for idx, class_name in enumerate(class_names):
    print(f"{idx:<2} => {class_name}")

# Splitting data

In [ ]:
# Separating data into training set and validation/test sets
(
training_texts, 
validation_testing_texts, 
training_labels, 
validation_testing_labels
) = train_test_split(
    texts_data,
    labels_data_encoded,
    train_size=0.8,
    random_state=RANDOM_STATE,
    stratify=labels_data_encoded,
)

# Separating validation and test sets
(
validation_texts, 
testing_texts, 
validation_labels, 
testing_labels
) = train_test_split(
    validation_testing_texts,
    validation_testing_labels,
    train_size=0.5,
    random_state=RANDOM_STATE,
    stratify=validation_testing_labels,
)

In [ ]:
# Verifying the correctness of dimensions
assert (
    training_texts.shape[0] + validation_texts.shape[0] + testing_texts.shape[0] \
    == texts_data.shape[0]
)

# Displaying the number of objects in each set
training_texts.shape[0], validation_texts.shape[0], testing_texts.shape[0]

# Data vizualization

In [ ]:
def plot_target_structure(labels, fig_title="Title_1"):
    """Plotting the shares of Dataset labels."""
    # Computing the unique labels
    unique_labels = np.unique(labels)
    # Computing the number of objects within each class
    labels_count = np.bincount(labels)
    
    # Computing the shares of objects in each class
    n_obj = labels.shape[0]
    
    labels_info_share = pd.Series(
        labels_count, index=label_encoder.classes_
    ) / n_obj
    
    # Plotting a figure
    labels_info_share.plot(kind="bar")
    plt.xticks(rotation=0)
    plt.title(fig_title, fontsize=15)
    plt.xlabel("Class name")
    plt.ylabel("Proportion of objects")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_target_structure(
    labels=labels_data_encoded, fig_title="Full data set"
)

In [ ]:
plot_target_structure(
    labels=training_labels, fig_title="Training set"
)

In [ ]:
plot_target_structure(
    labels=validation_labels, fig_title="Validation set"
)

In [ ]:
plot_target_structure(
    labels=testing_labels, fig_title="Test set"
)

# Tokenization

In [ ]:
# Instantiating a BERT tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# Tokenizing the training examples
training_encodings = bert_tokenizer(
    list(training_texts),
    add_special_tokens=True,
    max_length=128,
    truncation=True, 
    padding="max_length",
)

# Tokenizing the validation examples
validation_encodings = bert_tokenizer(
    list(validation_texts),
    add_special_tokens=True,
    max_length=128,
    truncation=True, 
    padding="max_length",
)

# Tokenizing the testing examples
testing_encodings = bert_tokenizer(
    list(testing_texts),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding="max_length",
)

In [ ]:
def display_encodings_info(
    tokenizer, 
    encodings, 
    texts, 
    labels,
    text_idx
):
    """Shows the original, encoded and decoded texts."""
    # Displaying the original text
    text = texts[text_idx]
    print(f"Input text:\n{text}\n")
    
    # Displaying the language of the text
    lang_label = labels[text_idx]
    lang = class_names[lang_label]
    print(f"Language: {lang}\n")
    
    # Displaying the encoded text
    text_encoded = encodings["input_ids"][text_idx]
    print(f"Tokenized input text (encoded):\n{text_encoded}\n")
    
    # Displaying the decoded text
    text_decoded = tokenizer.convert_ids_to_tokens(text_encoded)
    print(f"Tokenized input text (decoded):\n{text_decoded}")

In [ ]:
display_encodings_info(
    tokenizer=bert_tokenizer,
    encodings=training_encodings, 
    texts=training_texts, 
    labels=training_labels,
    text_idx=990,
)

In [ ]:
display_encodings_info(
    tokenizer=bert_tokenizer,
    encodings=validation_encodings, 
    texts=validation_texts, 
    labels=validation_labels,
    text_idx=1033,
)

In [ ]:
display_encodings_info(
    tokenizer=bert_tokenizer,
    encodings=testing_encodings, 
    texts=testing_texts, 
    labels=testing_labels,
    text_idx=1010,
)

# Creating Datasets and Dataloaders

In [ ]:
class LanguageDataset(Dataset):
    """Class for creating a custom dataset."""
    
    def __init__(self, encodings, labels):
        """Constructor for LanguageDataset class."""
        self.encodings = encodings
        self.labels = labels
        
    def __len__(self):
        """Computes the number of the dataset objects."""
        dataset_length = len(self.labels)
        
        return dataset_length

    def __getitem__(self, idx):
        """Returns the corresponding samples for index given."""
        item = {key: torch.tensor(value[idx])
                for key, value in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        
        return item

In [ ]:
# Initializing the training dataset
training_dataset = LanguageDataset(
    encodings=training_encodings, 
    labels=training_labels,
)

# Initializing the validation dataset
validation_dataset = LanguageDataset(
    encodings=validation_encodings, 
    labels=validation_labels,
)

# Initializing the testing dataset
testing_dataset = LanguageDataset(
    encodings=testing_encodings, 
    labels=testing_labels,
)

In [ ]:
# Creating a training Dataloader
training_dataloader = DataLoader(
    training_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
)

# Creating a validation Dataloader
validation_dataloader = DataLoader(
    validation_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
)

# Creating a testing Dataloader
testing_dataloader = DataLoader(
    testing_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=False,
)

In [ ]:
print(f"Training data examples: {len(training_dataloader.dataset):,}")
print(f"Number of batches: {len(training_dataloader)}")
print(f"Batch size: {BATCH_SIZE}")

In [ ]:
print(f"Validation data examples: {len(validation_dataloader.dataset)}")
print(f"Number of batches: {len(validation_dataloader)}")
print(f"Batch size: {BATCH_SIZE}")

In [ ]:
print(f"Testing data examples: {len(testing_dataloader.dataset):,}")
print(f"Number of batches: {len(testing_dataloader)}")
print(f"Batch size: {BATCH_SIZE}")

# Defining the BERT model

In [ ]:
# Creating a mapping from predictions to label names
id2label_mappings = dict()
for i, name in enumerate(class_names):
    id2label_mappings[i] = name
id2label_mappings

In [ ]:
# Computing the number of classes
num_labels = len(class_names)

# Instantiating the BERT model
bert_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=num_labels, 
    id2label=id2label_mappings,
)

# Moving the model to DEVICE (GPU/CUDA)
bert_model.to(DEVICE)

# Defining the optimization algorithm
optimizer = torch.optim.Adam(bert_model.parameters(), lr=LEARNING_RATE)

In [ ]:
bert_model.eval()

# Computing accuracy

In [ ]:
def acc_score(model, dataloader, device=DEVICE):
    """Computes the accuracy score for a DataLoader."""
    # Preallocating counter variables
    correct_predictions, num_examples = 0, 0
    
    # Turning off computing gradients
    with torch.no_grad():
        
        # Iteratively computing accuracy score (batch by batch)
        for batch_idx, batch in enumerate(dataloader):
            
            # Selecting the batch data (encodings, attention mask, labels)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            # Using BERT to compute logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs["logits"]
            
            # Computing the predictions for labels
            predicted_labels = torch.argmax(logits, dim=1)
            
            # Computing the number of examples/correct predictions number
            num_examples += labels.size(0)
            correct_predictions += (predicted_labels == labels).sum()
    
    # Computing the final accuracy score
    accuracy_score = correct_predictions.float() / num_examples
        
    return accuracy_score

# Fine-tuning loop

In [ ]:
def train_bert_model(
    model, 
    optimizer, 
    training_dataloader, 
    validation_dataloader,
    accuracy_score_func=acc_score,
    epochs=2,
    batch_log_freq=100,
    device=DEVICE
):
    """Launches the fine-tuning of BERT."""
    # Starting the timer
    start_time = time.time()
    
    # Going through all epochs
    for epoch in range(epochs):
        
        # Setting the model in the training mode
        model.train()
        
        # Going through all batches
        for batch_idx, batch in enumerate(training_dataloader):
        
            # Selecting the batch
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # BERT forward pass
            outputs = model(
                input_ids, attention_mask=attention_mask, labels=labels
            )
            loss, logits = outputs["loss"], outputs["logits"]
        
            # BERT backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Logging the progress
            if not batch_idx % batch_log_freq:
                print (f"Epoch {epoch+1:03d}/{epochs:03d} | "
                       f"Batch {batch_idx:03d}/{len(training_dataloader):03d} | "
                       f"Loss = {loss:.4f}")
        
        # Setting the model in the evaluation mode
        model.eval()
        
        # Disabling computing gradients
        with torch.set_grad_enabled(False):
            # Computing training accuracy
            training_accuracy_score = accuracy_score_func(
                model=model,
                dataloader=training_dataloader,
            )
            # Computing validation accuracy
            validation_accuracy_score = accuracy_score_func(
                model=model,
                dataloader=validation_dataloader,
            )
            # Logging the accuracy scores
            print(f"\nTraining accuracy = "
                  f"{training_accuracy_score:.4f}"
                  f"\nValid accuracy = "
                  f"{validation_accuracy_score:.4f}\n")
        
        # Printing the time passed at the end of the epoch
        time_elapsed_epoch = (time.time() - start_time) / 60
        print(f'Time elapsed: {time_elapsed_epoch:.2f} min\n')
    
    # Printing the total time spent on BERT fine-tuning
    time_elapsed_total = (time.time() - start_time) / 60
    print(f'\nTotal training Time: {time_elapsed_total:.2f} min')
    
    return model

In [ ]:
# Training the BERT model
bert_model = train_bert_model(
    model=bert_model,
    optimizer=optimizer,
    training_dataloader=training_dataloader,
    validation_dataloader=validation_dataloader,
    epochs=NUM_EPOCHS,
    batch_log_freq=10,
)

In [ ]:
def evaluate_test(model, dataloader, device=DEVICE):
    """Predicts the labels for the DataLoader."""
    # Setting up counter variables
    correct_preds, num_examples = 0, 0
    # Preallocating the list for test predictions
    test_predictions = []
    
    # Disabling computing gradients
    with torch.no_grad():
        
        # Iterating through all batches
        for batch_idx, batch in enumerate(dataloader):
            
            # Selecting the batch
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            # Computing logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs["logits"]
            
            # Computing the predictions for labels
            predicted_labels_batch = torch.argmax(logits, dim=1)
            
            # Adding the batch predictions to the list
            test_predictions.append(predicted_labels_batch)
            
            # Iteratively computing accuracy determinants
            num_examples += labels.size(0)
            correct_preds += (predicted_labels_batch == labels).sum().cpu()
    
    # Computing final accuracy score
    test_accuracy_score = correct_preds.float() / num_examples

    # Transforming a list of tensors into one tensor
    test_predictions_tensor = torch.cat(test_predictions).cpu()
            
    return test_accuracy_score, test_predictions_tensor

In [ ]:
# Computing test accuracy and test predictions
accuracy_test, predictions_test = evaluate_test(
    model=bert_model, dataloader=testing_dataloader
)

print(f"Test accuracy: {accuracy_test:.4f}")

# Testing the fine-tuned model on new data

In [ ]:
# Defining the Transformers pipeline
bert_pipeline = pipeline(
    task="text-classification",
    model=bert_model,
    tokenizer=bert_tokenizer,
    device=DEVICE,
)

## Predicting one text

In [ ]:
def define_lang_one_text(pipeline, test_text):
    """Outputs the language prediction for a sample text."""
    # Applying the pipeline to make predictions
    one_text_results = pipeline(test_text)[0]
    # Retrieving the probability
    proba = one_text_results["score"]
    # Retrieving the predicted label (encoded)
    predicted_lang = one_text_results["label"]
    # Displaying the prediction
    print(f"Predicted language: {predicted_lang} ({proba:.2%} probability)")

In [ ]:
# Creating a test text (English)
test_text = "It is interesting how we all turned out to be in the same place."
print(f"Input text: {test_text}\n")

define_lang_one_text(pipeline=bert_pipeline, test_text=test_text)

In [ ]:
# Creating a test text (Dutch)
test_text = "Ik kan het zien. Dankuwel!"
print(f"Input text: {test_text}\n")

define_lang_one_text(pipeline=bert_pipeline, test_text=test_text)

In [ ]:
# Creating a test text (Spanish)
test_text = "Si yo fuera Maradona viviría como él"
print(f"Input text: {test_text}\n")

define_lang_one_text(pipeline=bert_pipeline, test_text=test_text)

In [ ]:
# Creating a test text (German)
test_text = "Ich habe keine zeit. Auf Wiedersehen!"
print(f"Input text: {test_text}\n")

define_lang_one_text(pipeline=bert_pipeline, test_text=test_text)

In [ ]:
# Creating a test text (Russian)
test_text = "Что бы я делал без тебя, я не знаю"
print(f"Input text: {test_text}\n")

define_lang_one_text(pipeline=bert_pipeline, test_text=test_text)

In [ ]:
# Creating a test text (Swedish)
test_text = "Ursäkta mig! Hej då!"
print(f"Input text: {test_text}\n")

define_lang_one_text(pipeline=bert_pipeline, test_text=test_text)

In [ ]:
# Creating a test text (Portuguese)
test_text = "Como chego à estação de trem? Como chego ao ponto de ônibus?"
print(f"Input text: {test_text}\n")

define_lang_one_text(pipeline=bert_pipeline, test_text=test_text)

## Predicting an array of texts

In [ ]:
# Creating sample texts for classification
test_texts = [
    "It is impossible to do without a dictionary.",
    "Из любой ситуации всегда есть выход. Нужно лишь его увидеть!",
    "Eu gostaria de reservar um quarto, por favor",
    "Hooplijk schrift zij niets negatiefs in haar recensie...",
    "Hvor er der en pengeautomat?",
    "Καλά, ευχαριστώ, Χάρηκα",
    "Tünaydın!",
    "Weisst was ich meine?",
    "Interesting! I will look it up!",
]

# Applying the pipeline on all test texts
multiple_texts_results = bert_pipeline(test_texts)

# Outputting the results as DataFrame
multiple_texts_df = pd.DataFrame(multiple_texts_results)
multiple_texts_df["test_text"] = test_texts
multiple_texts_df["predicted_lang"] = multiple_texts_df["label"].copy()

# Rearranging the columns
multiple_texts_df[["test_text", "predicted_lang", "score"]]

# Vizualizing the predictions

In [ ]:
print(classification_report(
    y_true=testing_labels, 
    y_pred=predictions_test, 
    target_names=class_names,
)
     )

In [ ]:
ConfusionMatrixDisplay.from_predictions(
    testing_labels, 
    predictions_test,
    display_labels=class_names,
    cmap="Blues",
)
plt.xticks(rotation=90)
plt.xlabel("Predicted language")
plt.ylabel("True language")
plt.title("Language recognition confusion matrix", fontsize=15)
plt.tight_layout()
plt.show()